<img src="https://deecamp.chuangxin.com/assets/image/logo_nav_zh.jpg" width="40%">

In [3]:
import pandas as pd
import csv

In [16]:
! ls /data/jupyter_root/ZJ/test1.0 

block_1.attributes  block_2.attributes	block_3.attributes
block_1.tuples	    block_2.tuples	block_3.tuples


## 定义一个函数，先输出原始feature文件的P & R，再输出通过D-Cube分块的P & R

名称 | 定义
 --- | ---
TP	| 真实类别为positive，模型预测的类别也为positive
FP	| 预测为positive，但真实类别为negative，真实类别和预测类别不一致
FN	| 预测为negative，但真实类别为positive，真实类别和预测类别不一致
TN	| 真实类别为negative，模型预测的类别也为negative

准确率（accuracy）计算公式如下所示 不计算这个

\begin{equation}\label{equ:accuracy} \mbox{accuracy} = \frac{TP+TN}{TP+TN+FP+FN} = \frac{TP+TN }{\mbox{all data}} \end{equation}

positive class的精确率（precision）计算公式如下

\begin{equation}\label{equ:precision} \mbox{precision} = \frac{TP}{TP+FP} = \frac{TP}{\mbox{预测为positive的样本}} \end{equation}

positive class的召回率（recall）计算公式如下

\begin{equation}\label{equ:recall} \mbox{recall} = \frac{TP}{TP+FN} = \frac{TP}{\mbox{真实为positive的样本}} \end{equation}

In [22]:
def get_PR(f_count,choice,j=3):#f_count：the index of feature;j: index of test; choice: index of block
    
    #准备用于读取数据的 pandas头（防止第一行数据成为标签）
    data_path="/data/jupyter_root/dcube_data/raw_have_name/feature"#feature文件位置
    #try:
    cols = pd.read_csv(data_path+str(f_count)+"/block_1.tuples",nrows=1).columns#先读第一行，用于提取数据维度
    #except e:
    #    print(e)
    #    print("false1")
    #    return 0        

    user_index=['user_name']#初始化一个等待补全的names
    #print(cols.shape[0]-1)
    for i in range(1,cols.shape[0]-1):
            user_index.append(str(i))
    user_index.append('count')#补上最后的count
    
    #开始读取数据
    #result=pd.read_csv(data_path+str(f_count)+"/test"+str(j)+"/block_1.tuples",sep=',',names=user_index)#读取全部的数据，并给上names
    #try:
    if(choice=='all'):#读取blocks.txt
        result=pd.read_csv(data_path+str(f_count)+"/blocks.txt",sep=',',names=user_index)
    elif(choice==1 or choice==2 or choice==3 or choice==4 or choice==5 ):#读取block_choice.tuples
        result=pd.read_csv(data_path+str(f_count)+"/block_"+str(choice)+".tuples",sep=',',names=user_index)
    elif(choice==12 or choice==13 or choice==23 or choice ==123 ):#读取blockchoice.txt
        result=pd.read_csv(data_path+str(f_count)+"/block"+str(choice)+".txt",sep=',',names=user_index)
    #except:
    #    print("false2")
    #    return 0
    names = result['user_name'].drop_duplicates()#预测出坏用户的用户列表
    #print(names)
    dataset = pd.read_csv(open('/data/csv/label_eventsV1.csv','r',encoding = 'gb18030'))#读取总标签数据
    data = dataset[dataset['user_name'].isin(names)][['user_name','label']]#预测出坏用户的真实标签分布
    data = data.drop_duplicates()#去重
    #print(data)

    
    raw_result=pd.read_csv(data_path+str(f_count)+".txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
    raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
    #print(raw_names)
    raw_data = dataset[dataset['user_name'].isin(raw_names)][['user_name','label']]#原始数据中坏用户的真实标签分布
    raw_data = raw_data.drop_duplicates()#去重
    #print(raw_data)
    
    
    TP=data.label.sum()
    #if(TP==0):
        #return 0
    P=raw_data.label.sum()
    FP=data.label.count()-data.label.sum()
    N=raw_data.label.count()-raw_data.label.sum()
    Precision = (TP/P)/(TP/P+FP/N) #精确率计算

    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
    Recall = data.label.sum()/raw_data.label.sum() # 召回率计算
    print(len(names),len(raw_names))
    print("TP",TP,"P",P,"FP",FP,"N",N)
    print('测试特征'+str(f_count)+"_"+str(j)+str(choice))
    print('\t\t精确率 Precision\t= '+str(Precision))
    print('\t\t召回率 Recall   \t= '+str(Recall))
#get_PR(1,1)

In [28]:
def get_no_name_PR(f_count,feature_user,find_user):
    
    dataset = pd.read_csv(open('/data/csv/label_eventsV1.csv','r',encoding = 'gb18030'))#读取总标签数据
    data = dataset[dataset['user_name'].isin(find_user)][['user_name','label']]#预测出坏用户的真实标签分布
    data = data.drop_duplicates()#去重
    #print(data)

    
    #raw_result=pd.read_csv(data_path+str(f_count)+".txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
    #raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
    #print(raw_names)
    raw_data = dataset[dataset['user_name'].isin(feature_user)][['user_name','label']]#原始数据中坏用户的真实标签分布
    raw_data = raw_data.drop_duplicates()#去重
    #print(raw_data)
    
    
    TP=data.label.sum()
    #if(TP==0):
        #return 0
    P=raw_data.label.sum()
    FP=data.label.count()-data.label.sum()
    N=raw_data.label.count()-raw_data.label.sum()
    Precision = (TP/P)/(TP/P+FP/N) #精确率计算

    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
    Recall = data.label.sum()/raw_data.label.sum() # 召回率计算
    print(len(feature_user),len(find_user))
    print("TP",TP,"P",P,"FP",FP,"N",N)
    print('测试特征'+f_count)
    print('\t\t精确率 Precision\t= '+str(Precision))
    print('\t\t召回率 Recall   \t= '+str(Recall))

In [36]:
import re
result_list=pd.read_csv("/data/jupyter_root/dcube_data/raw_no_name/result_list.csv")
#print(result_list)
def get_set(s):
    s=re.sub('[ {}]','',s).split(',')
    def func(x):
        return int(float(x))
    return set(map(func,s))
for i in range(12):
    #try:
        al=get_set(result_list.iloc[i,0])
        get_no_name_PR("feature"+str(i)+'block_1',al,get_set(result_list.iloc[i,1]))
        get_no_name_PR("feature"+str(i)+'block_2',al,get_set(result_list.iloc[i,2]))
        get_no_name_PR("feature"+str(i)+'block_3',al,get_set(result_list.iloc[i,3]))
        get_no_name_PR("feature"+str(i)+'block_4',al,get_set(result_list.iloc[i,4]))
        get_no_name_PR("feature"+str(i)+'block_5',al,get_set(result_list.iloc[i,5]))
    #except:
        #print('false')
        #continue
#for i in range

1135368 33168
TP 219 P 5538 FP 148 N 3871
测试特征feature0block_1
		精确率 Precision	= 0.5084339257022874
		召回率 Recall   	= 0.03954496208017335
1135368 31481
TP 176 P 5538 FP 143 N 3871
测试特征feature0block_2
		精确率 Precision	= 0.4624505338609721
		召回率 Recall   	= 0.03178042614662333
1135368 31320
TP 221 P 5538 FP 148 N 3871
测试特征feature0block_3
		精确率 Precision	= 0.510705832136898
		召回率 Recall   	= 0.03990610328638498
1135368 30959
TP 199 P 5538 FP 139 N 3871
测试特征feature0block_4
		精确率 Precision	= 0.5001775846026683
		召回率 Recall   	= 0.03593355001805706
1135368 29823
TP 192 P 5538 FP 141 N 3871
测试特征feature0block_5
		精确率 Precision	= 0.48765624077318265
		召回率 Recall   	= 0.03466955579631636
1135368 12874
TP 93 P 5538 FP 66 N 3871
测试特征feature1block_1
		精确率 Precision	= 0.4962061223055198
		召回率 Recall   	= 0.016793066088840736
1135368 12401
TP 86 P 5538 FP 52 N 3871
测试特征feature1block_2
		精确率 Precision	= 0.5361823985878154
		召回率 Recall   	= 0.015529071867100036
1135368 12238
TP 65 P 5538 FP 64 N 3871
测试特

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


1118211 4
TP 0 P 5484 FP 0 N 3828
测试特征feature9block_1
		精确率 Precision	= nan
		召回率 Recall   	= 0.0
1118211 77556
TP 858 P 5484 FP 361 N 3828
测试特征feature9block_2
		精确率 Precision	= 0.6239231875699544
		召回率 Recall   	= 0.15645514223194748
1118211 228238
TP 1515 P 5484 FP 839 N 3828
测试特征feature9block_3
		精确率 Precision	= 0.5576099447564808
		召回率 Recall   	= 0.2762582056892779
1118211 577342
TP 3698 P 5484 FP 2074 N 3828
测试特征feature9block_4
		精确率 Precision	= 0.5544879387820333
		召回率 Recall   	= 0.674325309992706
1118211 3
TP 0 P 5484 FP 0 N 3828
测试特征feature9block_5
		精确率 Precision	= nan
		召回率 Recall   	= 0.0
1118213 99374
TP 707 P 5484 FP 432 N 3828
测试特征feature10block_1
		精确率 Precision	= 0.5332291462252664
		召回率 Recall   	= 0.1289204959883297
1118213 240786
TP 1677 P 5484 FP 1027 N 3828
测试特征feature10block_2
		精确率 Precision	= 0.5326714474331427
		召回率 Recall   	= 0.3057986870897155
1118213 18709
TP 215 P 5484 FP 138 N 3828
测试特征feature10block_3
		精确率 Precision	= 0.5209607219086828
		召回率 Recall  

In [42]:
import re
result_list=pd.read_csv("/data/jupyter_root/dcube_data/raw_no_name/result_list_feature4.csv")
#print(result_list)
def get_set(s):
    s=re.sub('[ {}]','',s).split(',')
    def func(x):
        return int(float(x))
    return set(map(func,s))
i=0
al=get_set(result_list.iloc[i,0])
for x in range(1,21):
    get_no_name_PR("feature"+str(i)+'block_'+str(x),al,get_set(result_list.iloc[i,x]))

1118213 875
TP 12 P 5484 FP 3 N 3828
测试特征feature0block_1
		精确率 Precision	= 0.7362954414310445
		召回率 Recall   	= 0.002188183807439825
1118213 835
TP 25 P 5484 FP 1 N 3828
测试特征feature0block_2
		精确率 Precision	= 0.9458017077798861
		召回率 Recall   	= 0.004558716265499635
1118213 820
TP 16 P 5484 FP 3 N 3828
测试特征feature0block_3
		精确率 Precision	= 0.7882625482625483
		召回率 Recall   	= 0.0029175784099197666
1118213 805
TP 20 P 5484 FP 3 N 3828
测试特征feature0block_4
		精确率 Precision	= 0.823119597471294
		召回率 Recall   	= 0.0036469730123997084
1118213 802
TP 26 P 5484 FP 4 N 3828
测试特征feature0block_5
		精确率 Precision	= 0.8194032799841929
		召回率 Recall   	= 0.004741064916119621
1118213 791
TP 19 P 5484 FP 2 N 3828
测试特征feature0block_6
		精确率 Precision	= 0.8689605734767025
		召回率 Recall   	= 0.003464624361779723
1118213 772
TP 22 P 5484 FP 4 N 3828
测试特征feature0block_7
		精确率 Precision	= 0.7933529278770065
		召回率 Recall   	= 0.0040116703136396795
1118213 649
TP 20 P 5484 FP 3 N 3828
测试特征feature0block_8
		精确率 Prec

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


In [39]:
pd.read_csv("/data/jupyter_root/dcube_data/raw_have_name/feature_list.csv")

,Unnamed: 0,Feature_combination,Standard,Poly
0,feature1,"['user_name', 'time_stamp_3hour']",resource_type,nunique
1,feature2,"['user_name', 'time_stamp_3hour']",resource_category,nunique
2,feature3,"['user_name', 'ip_123', 'time_stamp_3hour']",index,count
3,feature4,"['user_name', 'user_agent', 'time_stamp_hour',...",index,count
4,feature5,"['user_name', 'os_version', 'time_stamp_hour',...",index,count
5,feature6,"['user_name', 'os_version', 'time_stamp_hour']",index,count
6,feature7,"['user_name', 'user_agent', 'time_stamp_3hour']",index,count
7,feature8,"['user_name', 'os_version', 'time_stamp_hour']",index,count
8,feature9,"['user_name', 'ip_123', 'time_stamp_6hour', 'e...",index,count
9,feature10,['user_name'],user_agent,nunique


In [37]:
pd.read_csv("/data/jupyter_root/dcube_data/raw_no_name/feature_list.csv")

,Unnamed: 0,Feature_combination,Standard,Poly
0,feature1,['time_stamp_3hour'],user_name,nunique
1,feature2,['time_stamp_hour'],user_agent,nunique
2,feature3,['time_stamp_3hour'],resource_owner,nunique
3,feature4,['ip_1234'],user_name,nunique
4,feature5,"['ip_123', 'user_agent']",user_name,nunique
5,feature6,"['ip_123', 'os_version']",user_name,nunique
6,feature7,"['ip_123', 'user_agent']",user_name,count
7,feature8,"['ip_123', 'os_version']",user_name,count
8,feature9,"['ip_123', 'time_stamp_3hour']",user_name,nunique
9,feature10,"['ip_123', 'time_stamp_3hour']",user_name,count


In [23]:
for i in range(1,10):
    #for j in range(51,136):
        #j=3
        #try:
            #if(j==3):
                get_PR(i,1)
                get_PR(i,2)
                get_PR(i,3)
                get_PR(i,4)
                get_PR(i,5)
                #get_PR(i,j,'all')
                #get_PR(i,j,12)
                #get_PR(i,j,13)
                #get_PR(i,j,23)
                #get_PR(i,j,123)
     # except:
     #     print("false")
     #     continue

72263 1135368
TP 1104 P 5538 FP 634 N 3871
测试特征1_31
		精确率 Precision	= 0.5489739071992207
		召回率 Recall   	= 0.19934994582881907
16136 1135368
TP 276 P 5538 FP 164 N 3871
测试特征1_32
		精确率 Precision	= 0.5405144518847249
		召回率 Recall   	= 0.04983748645720477
358378 1135368
TP 3321 P 5538 FP 1764 N 3871
测试特征1_33
		精确率 Precision	= 0.5682123852406292
		召回率 Recall   	= 0.5996749729144095
5486 1135368
TP 82 P 5538 FP 61 N 3871
测试特征1_34
		精确率 Precision	= 0.48443623710396194
		召回率 Recall   	= 0.014806789454676778
128751 1135368
TP 1378 P 5538 FP 849 N 3871
测试特征1_35
		精确率 Precision	= 0.5315103626943005
		召回率 Recall   	= 0.24882629107981222
71176 1135368
TP 1096 P 5538 FP 616 N 3871
测试特征2_31
		精确率 Precision	= 0.5542987584047293
		召回率 Recall   	= 0.19790538100397256


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in double_scalars


5 1135368
TP 0 P 5538 FP 0 N 3871
测试特征2_32
		精确率 Precision	= nan
		召回率 Recall   	= 0.0
14611 1135368
TP 259 P 5538 FP 155 N 3871
测试特征2_33
		精确率 Precision	= 0.5387427800098765
		召回率 Recall   	= 0.04676778620440592
361925 1135368
TP 3373 P 5538 FP 1799 N 3871
测试特征2_34
		精确率 Precision	= 0.5672036332287781
		召回率 Recall   	= 0.6090646442759119
4226 1135368
TP 68 P 5538 FP 55 N 3871
测试特征2_35
		精确率 Precision	= 0.4635781183407359
		召回率 Recall   	= 0.012278801011195377
2 1118211
TP 0 P 5484 FP 0 N 3828
测试特征3_31
		精确率 Precision	= nan
		召回率 Recall   	= 0.0
1 1118211
TP 0 P 5484 FP 0 N 3828
测试特征3_32
		精确率 Precision	= nan
		召回率 Recall   	= 0.0
1 1118211
TP 0 P 5484 FP 0 N 3828
测试特征3_33
		精确率 Precision	= nan
		召回率 Recall   	= 0.0
1 1118211
TP 0 P 5484 FP 0 N 3828
测试特征3_34
		精确率 Precision	= nan
		召回率 Recall   	= 0.0
2 1118211
TP 0 P 5484 FP 0 N 3828
测试特征3_35
		精确率 Precision	= nan
		召回率 Recall   	= 0.0
349513 1135368
TP 3923 P 5538 FP 2038 N 3871
测试特征4_31
		精确率 Precision	= 0.5736520373671016
		召回率 Rec

In [25]:
def get_PR_mini(f_count,choice,j=3):#f_count：the index of feature;j: index of test; choice: index of block
    
    #准备用于读取数据的 pandas头（防止第一行数据成为标签）
    data_path="/data/jupyter_root/dcube_data/mini_have_name/feature"#feature文件位置
    #try:
    cols = pd.read_csv(data_path+str(f_count)+"/block_1.tuples",nrows=1).columns#先读第一行，用于提取数据维度
    #except e:
    #    print(e)
    #    print("false1")
    #    return 0        

    user_index=['user_name']#初始化一个等待补全的names
    #print(cols.shape[0]-1)
    for i in range(1,cols.shape[0]-1):
            user_index.append(str(i))
    user_index.append('count')#补上最后的count
    
    #开始读取数据
    #result=pd.read_csv(data_path+str(f_count)+"/test"+str(j)+"/block_1.tuples",sep=',',names=user_index)#读取全部的数据，并给上names
    #try:
    if(choice=='all'):#读取blocks.txt
        result=pd.read_csv(data_path+str(f_count)+"/blocks.txt",sep=',',names=user_index)
    elif(choice==1 or choice==2 or choice==3 or choice==4 or choice==5 ):#读取block_choice.tuples
        result=pd.read_csv(data_path+str(f_count)+"/block_"+str(choice)+".tuples",sep=',',names=user_index)
    elif(choice==12 or choice==13 or choice==23 or choice ==123 ):#读取blockchoice.txt
        result=pd.read_csv(data_path+str(f_count)+"/block"+str(choice)+".txt",sep=',',names=user_index)
    #except:
    #    print("false2")
    #    return 0
    names = result['user_name'].drop_duplicates()#预测出坏用户的用户列表
    #print(names)
    dataset = pd.read_csv(open('/data/csv/label_eventsV1.csv','r',encoding = 'gb18030'))#读取总标签数据
    data = dataset[dataset['user_name'].isin(names)][['user_name','label']]#预测出坏用户的真实标签分布
    data = data.drop_duplicates()#去重
    #print(data)

    
    raw_result=pd.read_csv(data_path+str(f_count)+".txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
    raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
    #print(raw_names)
    raw_data = dataset[dataset['user_name'].isin(raw_names)][['user_name','label']]#原始数据中坏用户的真实标签分布
    raw_data = raw_data.drop_duplicates()#去重
    #print(raw_data)
    
    
    TP=data.label.sum()
    #if(TP==0):
        #return 0
    P=raw_data.label.sum()
    FP=data.label.count()-data.label.sum()
    N=raw_data.label.count()-raw_data.label.sum()
    Precision = (TP/P)/(TP/P+FP/N) #精确率计算

    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
    Recall = data.label.sum()/raw_data.label.sum() # 召回率计算
    print(len(names),len(raw_names))
    print("TP",TP,"P",P,"FP",FP,"N",N)
    print('测试特征'+str(f_count)+"_"+str(j)+str(choice))
    print('\t\t精确率 Precision\t= '+str(Precision))
    print('\t\t召回率 Recall   \t= '+str(Recall))

In [26]:
for i in range(1,10):
    #for j in range(51,136):
        #j=3
        #try:
            #if(j==3):
                get_PR_mini(i,1)
                get_PR_mini(i,2)
                get_PR_mini(i,3)
                get_PR_mini(i,4)
                get_PR_mini(i,5)

1344 9409
TP 848 P 5538 FP 496 N 3871
测试特征1_31
		精确率 Precision	= 0.5444285520949155
		召回率 Recall   	= 0.1531238714337306
1949 9409
TP 1282 P 5538 FP 667 N 3871
测试特征1_32
		精确率 Precision	= 0.5732848547467628
		召回率 Recall   	= 0.23149151318165404
3 9409
TP 2 P 5538 FP 1 N 3871
测试特征1_33
		精确率 Precision	= 0.5829819277108435
		召回率 Recall   	= 0.00036114120621162876
4450 9409
TP 2880 P 5538 FP 1570 N 3871
测试特征1_34
		精确率 Precision	= 0.5618304361104139
		召回率 Recall   	= 0.5200433369447454
3 9409
TP 2 P 5538 FP 1 N 3871
测试特征1_35
		精确率 Precision	= 0.5829819277108435
		召回率 Recall   	= 0.00036114120621162876
1772 9409
TP 1152 P 5538 FP 620 N 3871
测试特征2_31
		精确率 Precision	= 0.5649840515943845
		召回率 Recall   	= 0.20801733477789816
1 9409
TP 0 P 5538 FP 1 N 3871
测试特征2_32
		精确率 Precision	= 0.0
		召回率 Recall   	= 0.0
1 9409
TP 0 P 5538 FP 1 N 3871
测试特征2_33
		精确率 Precision	= 0.0
		召回率 Recall   	= 0.0
383 9409
TP 232 P 5538 FP 151 N 3871
测试特征2_34
		精确率 Precision	= 0.5178266861172456
		召回率 Recall   	= 0.041

In [51]:
#特征取交集测试
get_PR(57,3,1)
get_PR(57,3,2)
import pandas as pd
dataset = pd.read_csv(open('/data/csv/label_eventsV1.csv','r',encoding = 'gb18030'))#读取总标签数据
user_index=['user_name']#初始化一个等待补全的names
#print(cols.shape[0]-1)
for i in range(1,3):
        user_index.append(str(i))
user_index.append('count')
user_index.append('now')
result_1=pd.read_csv("/data/jupyter_root/dcube_data/feature57/block_1.tuples",sep=',',names=user_index)

#user_index.pop()
result_2=pd.read_csv("/data/jupyter_root/dcube_data/feature57/block_2.tuples",sep=',',names=user_index)
names=set(result_1['user_name'].drop_duplicates().values)&set(result_2['user_name'].drop_duplicates().values)
data = dataset[dataset['user_name'].isin(names)][['user_name','label']]#预测出坏用户的真实标签分布
data = data.drop_duplicates()

#user_index.append('now')
raw_result=pd.read_csv("/data/jupyter_root/dcube_data/feature57.txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
#print(raw_names)
raw_data = dataset[dataset['user_name'].isin(raw_names)][['user_name','label']]#原始数据中用户的真实标签分布
raw_data = raw_data.drop_duplicates()#去重

TP=data.label.sum()
P=raw_data.label.sum()
FP=data.label.count()-data.label.sum()
N=raw_data.label.count()-raw_data.label.sum()
Precision = (TP/P)/(TP/P+FP/N) #精确率计算


    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
Recall = data.label.sum()/raw_data.label.sum() # 召回率计算
print("TP",TP,"P",P,"FP",FP,"N",N)
print('\t\t精确率 Precision\t= '+str(Precision))
print('\t\t召回率 Recall   \t= '+str(Recall))
#user_index.pop()

raw_result=pd.read_csv("/data/jupyter_root/dcube_data/feature57.txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
#print(raw_names)
raw_data = dataset[dataset['user_name'].isin(raw_names)][['user_name','label']]#原始数据中坏用户的真实标签分布
raw_data = raw_data.drop_duplicates()#去重

TP=data.label.sum()
P=raw_data.label.sum()
FP=data.label.count()-data.label.sum()
N=raw_data.label.count()-raw_data.label.sum()
Precision = (TP/P)/(TP/P+FP/N) #精确率计算

    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
Recall = data.label.sum()/raw_data.label.sum() # 召回率计算
print("TP",TP,"P",P,"FP",FP,"N",N)
print('\t\t精确率 Precision\t= '+str(Precision))
print('\t\t召回率 Recall   \t= '+str(Recall))

TP 3244 P 5538 FP 1666 N 3871
测试特征57_31
		精确率 Precision	= 0.5764607439131921
		召回率 Recall   	= 0.5857710364752619
TP 2431 P 5538 FP 2186 N 3871
测试特征57_32
		精确率 Precision	= 0.437358053498462
		召回率 Recall   	= 0.43896713615023475
TP 172 P 5538 FP 160 N 3871
		精确率 Precision	= 0.4290324326692837
		召回率 Recall   	= 0.031058143734200072
TP 172 P 5538 FP 160 N 3871
		精确率 Precision	= 0.4290324326692837
		召回率 Recall   	= 0.031058143734200072


# 精确率Precision的计算

# 召回率Recall的计算

In [5]:
pd.set_option('max_row',200) 
pd.set_option('display.max_colwidth', 100)
pd.read_csv("/data/jupyter_root/dcube_data/have_name/feature_list.csv",sep=',')

,Unnamed: 0,Feature_combination,Standard,Poly,user_name_dic,user_name_num
0,feature1,['user_name'],user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
1,feature2,"['user_name', 'time_stamp_day']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
2,feature3,"['user_name', 'time_stamp_hour']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
3,feature4,"['user_name', 'time_stamp_3hour']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
4,feature5,"['user_name', 'time_stamp_6hour']",user_agent,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
5,feature6,['user_name'],os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
6,feature7,"['user_name', 'time_stamp_day']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
7,feature8,"['user_name', 'time_stamp_hour']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
8,feature9,"['user_name', 'time_stamp_3hour']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409
9,feature10,"['user_name', 'time_stamp_6hour']",os_version,nunique,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...",9409


In [61]:
pwd

'/data/jupyter_root/LSK'

In [21]:
import pandas as pd
user_index=['user_name']#初始化一个等待补全的names
#print(cols.shape[0]-1)
for i in range(1,3):
        user_index.append(str(i))
user_index.append('count')
result_1=pd.read_csv("./block12.txt",sep=',',names=user_index)
result_2=pd.read_csv("./block13.txt",sep=',',names=user_index)
set(result_1['user_name'].drop_duplicates().values)&set(result_2['user_name'].drop_duplicates().values)
fet

{1343494,
 458762,
 1310734,
 1343510,
 1540121,
 1671197,
 65569,
 32805,
 65598,
 524351,
 393281,
 524359,
 1540168,
 1638472,
 524364,
 65614,
 1540176,
 393300,
 65629,
 1146980,
 262247,
 196712,
 65645,
 360561,
 262258,
 1540214,
 1605754,
 229504,
 1409157,
 393356,
 1540237,
 1310879,
 1376418,
 1376419,
 1540260,
 262312,
 1671346,
 1179832,
 65722,
 1179837,
 65728,
 393420,
 491735,
 1147095,
 1540313,
 1147101,
 295139,
 1343718,
 1179879,
 327915,
 65779,
 327928,
 262394,
 524541,
 295167,
 1540356,
 1343750,
 1540359,
 1409286,
 1179919,
 1540369,
 65821,
 1507622,
 1540391,
 1311025,
 1179954,
 1147187,
 459060,
 1212727,
 1179962,
 393538,
 1507655,
 1147208,
 1179977,
 1507657,
 1507668,
 1507673,
 1540452,
 1507685,
 1540453,
 1671524,
 1507689,
 1507693,
 98670,
 1507698,
 1343865,
 196993,
 1507716,
 1507719,
 98696,
 524681,
 1507724,
 1573265,
 1507730,
 655763,
 197018,
 131491,
 1540516,
 393640,
 524723,
 1507763,
 1507769,
 131514,
 491981,
 1507792,
 15077

In [27]:
pd.set_option('max_row',200) 
pd.set_option('display.max_colwidth', 100)
pd.read_csv("/data/jupyter_root/dcube_data/no_name/feature_list.csv",sep=',')

,Unnamed: 0,Feature_combination,Standard,Poly
0,feature1,['time_stamp_day'],user_agent,nunique
1,feature2,['time_stamp_hour'],user_agent,nunique
2,feature3,['time_stamp_3hour'],user_agent,nunique
3,feature4,['time_stamp_6hour'],user_agent,nunique
4,feature5,['time_stamp_day'],os_version,nunique
5,feature6,['time_stamp_hour'],os_version,nunique
6,feature7,['time_stamp_3hour'],os_version,nunique
7,feature8,['time_stamp_6hour'],os_version,nunique
8,feature9,['time_stamp_day'],ip_1,nunique
9,feature10,['time_stamp_hour'],ip_1,nunique


In [12]:
pd.set_option('max_row',200) 
pd.set_option('display.max_colwidth', 100)
pd.read_csv("/data/jupyter_root/dcube_data/no_name_add/feature_list.csv",sep=',')

,Unnamed: 0,Feature_combination,Standard,Poly
0,feature126,['user_agent'],user_name,nunique
1,feature127,"['user_agent', 'time_stamp_day']",user_name,nunique
2,feature128,"['user_agent', 'time_stamp_hour']",user_name,nunique
3,feature129,"['user_agent', 'time_stamp_3hour']",user_name,nunique
4,feature130,"['user_agent', 'time_stamp_6hour']",user_name,nunique
5,feature131,['os_version'],user_name,nunique
6,feature132,"['os_version', 'time_stamp_day']",user_name,nunique
7,feature133,"['os_version', 'time_stamp_hour']",user_name,nunique
8,feature134,"['os_version', 'time_stamp_3hour']",user_name,nunique
9,feature135,"['os_version', 'time_stamp_6hour']",user_name,nunique


In [7]:
pd.set_option('max_row',200) 
pd.set_option('display.max_colwidth', 100)
pd.read_csv("/data/jupyter_root/dcube_data/no_name_add/result_list2_811.csv",sep=',')

,feature_user,block_1.tuples,block_2.tuples,block_3.tuples,block_4.tuples,block_5.tuples
0,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 1671224, 655...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167..."
1,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 245766, 1597450, 319501, 1310734, 221217, 65569, 548900, 1130535, 41012, 1589314, 2212...","{1130498, 466948, 286725, 245766, 319501, 81934, 1310734, 1327123, 516120, 221211, 65569, 548900...","{1130498, 1089538, 466949, 499719, 1310734, 1646615, 65569, 1130535, 1089592, 114747, 1638472, 1...","{466948, 1658884, 546826, 1601547, 1310734, 1548306, 1529874, 118805, 1343510, 544789, 221211, 1...","{1595392, 1089538, 1658884, 245766, 499719, 221192, 348169, 546826, 141325, 1433614, 1556496, 12..."
2,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{20487, 221192, 1514503, 1182730, 76304, 1165329, 1548306, 282649, 521755, 1590812, 160289, 2207...","{280576, 373249, 389632, 102914, 395264, 160268, 319501, 1652756, 520214, 73243, 65569, 1583652,...","{198656, 73218, 117763, 114182, 1604102, 92681, 1573897, 1464845, 1290254, 453138, 1653782, 5207...","{1508352, 461314, 1343494, 245766, 118796, 1464845, 1313814, 471065, 73243, 250396, 118817, 6556...","{1576450, 1090563, 419333, 116742, 519687, 1524743, 118805, 116761, 542748, 475676, 366110, 2150..."
3,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{280576, 395264, 317450, 319501, 81934, 1308689, 1327123, 1652756, 118805, 520214, 1656859, 6556...","{1343494, 1536007, 245766, 1597450, 118796, 319501, 118805, 471065, 1607712, 118817, 65569, 5489...","{245766, 20487, 221192, 1165329, 1548306, 282649, 116788, 1071162, 520253, 1656894, 1052743, 577...","{116742, 1323024, 118805, 1646615, 116761, 542748, 1587229, 215072, 1634346, 479275, 550958, 107...","{198656, 245766, 1290254, 1298455, 215072, 460840, 1177644, 116788, 471108, 71753, 397389, 13067..."
4,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{280576, 1650688, 395264, 286725, 317450, 319501, 81934, 1308689, 1327123, 1652756, 118805, 5202...","{1595392, 243714, 1644547, 466948, 102406, 499719, 546826, 419854, 1290254, 1556498, 1327123, 52...","{1343494, 1536007, 245766, 317450, 1597450, 118796, 319501, 118805, 471065, 503836, 1607712, 118...","{1089538, 116742, 1323024, 118805, 1646615, 1595415, 116761, 542748, 1587229, 1585183, 282656, 2...","{1601547, 118805, 1343510, 544789, 102430, 1251361, 548900, 528420, 542765, 247858, 178232, 1308..."
5,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 245766, 1323028, 247832, 1308713, 1316907, 1212475, 348223, 471108, 1321029, 1052743, ...","{1343494, 245766, 1085451, 1316875, 1433614, 669711, 1323024, 1331217, 1310734, 1323028, 1333269...","{745476, 1343494, 1308689, 1327123, 563221, 1343510, 471065, 284700, 41012, 178232, 471108, 5776...","{1338373, 163336, 1540628, 1336853, 1326614, 1502744, 280095, 1212475, 1347137, 523330, 547399, ..."
6,"{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1343494, 458762, 1310734, 1343510, 1540121, 1671194, 1671197, 65569, 32805, 65582, 1146929, 167...","{1089538, 466948, 22